<a href="https://colab.research.google.com/github/vlamen/tue-deeplearning/blob/main/practicals/P1.2_Word_embedding_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# Practical Overview

 In this Practical you will learn :
- Basic concepts of word embeddings.
- How to extract word embeddings by N-Gram in a toy example.
- How to extract word embeddings by CBoW in a toy example.
- How to extract word embeddings by CBoW in `Alice in Wonderland`.
- How to perform the analogy task.
- Reflecting on the conceptual differences between the CBoW and N-Gram model.

We recommend running the practicals of this course in Google Colab. Make sure to go to additional connection options (the small arrow at the top right) -> change runtime type -> a GPU runtime to make use of GPU acceleration.



# Word Embeddings: Encoding Lexical Semantics


Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by
$$
\begin{align}
\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}
\end{align}
$$

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

- We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the [distributional
hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics).


# Getting Dense Word Embeddings


How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

$$
\begin{align}
q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]
 \end{align}
$$

$$
\begin{align}
q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]
\end{align}
$$

Then we can get a measure of similarity between these words by doing:

$$
\begin{align}
\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}
\end{align}
$$


Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.



You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch


Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).



In [ ]:
# The first part of this practical is based on the tutorial by Robert Guthrie, which can be found here: 
# https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)

In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

# Getting started:  N-Gram Language Modeling in a Toy example


Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.




##  Data Preprocessing

Before we can use the data in our model, we should build a list of tuples.  Each tuple is `([ word_i-2, word_i-1 ], target word)`, we also build dictionaries to map from words to indices, i.e. word_to_ix, and map from indices to words, i.e. ix_to_word,

In [ ]:
CONTEXT_SIZE = 2   # a 2-gram language model

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])


vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}


## Create N-Gram model
We create N-Gram language model, and set the word embedding size to be 100 (or other by your preference).

In [ ]:
EMBEDDING_DIM = 100

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs



loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(100):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    average_loss= total_loss/len(trigrams)
    print('Epoch:', epoch, '\tLoss:', average_loss)  # The loss decreased every iteration over the training data!

##  Check the results

We can inspect some of the word embedding values in a pandas DataFrame:

In [ ]:
import pandas as pd
embeddings = model.embeddings(torch.Tensor([list(range(0,len(vocab)))]).long())
embeddings = embeddings.view(-1,100)
pd.DataFrame(embeddings.tolist(), index=list(ix_to_word.values())[0:]).head()

We can also plot the embeddings with TSNE to visualize the similarity

In [ ]:
labels=[]
for i in range(len(ix_to_word)):
    labels.append(ix_to_word[i])

In [ ]:
from matplotlib import pylab
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
two_d_embeddings = tsne.fit_transform(embeddings.detach().numpy())

def plot(embeddings, labels):
      assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
      pylab.figure(figsize=(10,10))  # in inches
      for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                       ha='right', va='bottom')
      pylab.show()


plot(two_d_embeddings, labels)

In [ ]:
context = ['When','forty']

a = model(torch.tensor([word_to_ix[w] for w in context]))

#Print result
print(f'Raw text: {" ".join(test_sentence)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

# Practical part  1: Computing Word Embeddings -- Continuous Bag-of-Words in a Toy example


The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from typical practices in language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




##  Data Preprocessing

The CBOW model architecture tries to predict the current target word (the center word) based on the source context words (surrounding words). Considering a simple sentence, `the quick brown fox jumps over the lazy dog`, this can be pairs of (context_window, target_word) where if we consider a context size 2 (2 words to the left, 2 to the right),  we have examples like `([the, quick, fox, jumps], brown)` and so on. Thus the model tries to predict the target_word based on the context_window words.


Therefore should build a list of tuples.  Each tuple is `([ word_i-2, word_i-1,word_i+1, word_i+2 ], target word)`, we also build dictionaries to map from words to indices, i.e. word_to_ix, and map from indices to words, i.e. ix_to_word,

In [ ]:
import torch
import torch.nn as nn


CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 100

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


# By deriving a set from `raw_text`, we deduplicate the list
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
    
print(data[:5])


## Create CBoW Model
Following is an image to illustrate the model architecture. We set the word embedding size to be 100 (or other by your preference).

![avatar](https://miro.medium.com/max/328/1*bkrBASpteKfCaxZDEEeN6g.png)

In [ ]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, vocab_size)
        # self.activation_function1 = nn.ReLU()
        
        # #out: 1 x vocab_size
        # self.linear2 = nn.Linear(128, vocab_size)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)

        out = self.linear1(embeds)
        # you can also try playing around with an additional hidden layer after the summation
        # out = self.activation_function1(out)  
        
        # out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


# create your model and train. Here are some functions to help you make
# the data ready for use by your module

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)



model = CBOW(vocab_size, EMBEDDING_DIM)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)




#TRAINING
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_vector = make_context_vector(context, word_to_ix)  

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over near
        # words
        log_probs = model(context_vector)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    average_loss= total_loss/len(data)
    print('Epoch:', epoch, '\tLoss:', average_loss)  # The loss decreased

##  Check the results

Then we check the results by making predictions of target work by context

In [ ]:
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

# Practical part 2: Computing Word Embeddings in  `Alice in Wonderland` using CBoW

Now let's try to build word embedding in a ‘Alice in Wonderland’ instead of a toy example. The word-label mapping dictionaries and corpus are available [here](https://drive.google.com/drive/folders/1mYM29PbWQ4rS0bgKndFJjZlkbcwMaEpO?usp=sharing). You can use the cell bellow to download the necessary data in your working directory.

In [ ]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1mYM29PbWQ4rS0bgKndFJjZlkbcwMaEpO")

##  Data Preprocessing

In [ ]:
# modify the path according to your own file location
import os
data_path=os.path.join("/content/Word_embedding_data/")
# if you are not using Colab, change data_path above to where you stored the data locally

In [ ]:
# data_path=os.path.join(".")

Now we load the data:

In [ ]:
import json
with open(os.path.join(data_path,"alice.json"),'r') as f:
    alice = json.load(f)
with open(os.path.join(data_path,"ix_to_word.json"),'r') as f:
    ix_to_word = json.load(f)
with open(os.path.join(data_path,"word_to_ix.json"),'r') as f:
    word_to_ix = json.load(f)

Then we create the context and target pairs by borrowing some keras utilities. We set the window_size 2 (2 words to the left, 2 to the right).

In [ ]:
window_size=2
vocab_size=len(word_to_ix)

from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences

corpus = [[word_to_ix[w] for w in text.text_to_word_sequence(doc)] for doc in alice]

def generate_context_word_pairs(corpus, window_size, vocab_size):
    X = []
    Y = []
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):           
            start = index - window_size
            end = index + window_size + 1
            context = [words[i] for i in range(start, end)if 0 <= i < sentence_length and i != index]
            x = pad_sequences([context], maxlen=context_length)[0]
            X.append(x)
            Y.append([word])

    X=torch.tensor(X, dtype=torch.long)
    Y=torch.tensor(Y, dtype=torch.long)
    return X,Y

X,Y = generate_context_word_pairs(corpus, window_size=window_size, vocab_size=vocab_size)


##  Create CBoW Model
Now we create CBoW model, we could set the word embedding size to be 100 (or  some other number if you prefer).

In [ ]:
# Alternative model architecture with an additional hidden layer. This does not precisely adhere to the
# description of CBOW presented above:

# class CBOW(torch.nn.Module):

#     def __init__(self, vocab_size, embedding_dim=100):
#         super(CBOW, self).__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.linear1 = nn.Linear(embedding_dim, 100)
#         self.activation_function1 = nn.ReLU()        
#         self.linear2 = nn.Linear(100, vocab_size)
#         self.activation_function2 = nn.LogSoftmax(dim = -1)
        

#     def forward(self, inputs):
#         embeds = sum(self.embeddings(inputs)).view(1,-1)
#         out = self.linear1(embeds)
#         out = self.activation_function1(out)
#         out = self.linear2(out)
#         out = self.activation_function2(out)
#         return out
###
    
# standard CBoW architecture:

class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, vocab_size)
        # self.activation_function1 = nn.ReLU()
        
        # #out: 1 x vocab_size
        # self.linear2 = nn.Linear(128, vocab_size)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)

        out = self.linear1(embeds)
        # out = self.activation_function1(out)
        
        # out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
    
EMBEDDING_DIM=100
model = CBOW(vocab_size,EMBEDDING_DIM)

Remember to move your model to GPU, since now we are training with a larger corpus.

In [ ]:
# Move model to GPU

dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(dev)

Then we train the model and print the loss. The loss should decrease with every iteration over the training data. You can play with the epochs numbers here.

In [ ]:
#TRAINING

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)



for epoch in range(0, 10):
    total_loss = 0.
    i = 0

    for context, target in zip(X,Y):
        
        # Step 1. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 2. Run the forward pass, getting log probabilities.
        log_probs = model(context.to(dev))

        # Step 3. Compute your loss function. 
        loss = loss_function(log_probs,target.to(dev))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()


        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    average_loss= total_loss/len(X)
    print('Epoch:', epoch, '\tLoss:', average_loss)  # The loss decreased every iteration over the training data!
    
  

# Practical. 3: Analogy Task




**Reading material**
* [1] Mikolov, Tomas, et al. "[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)" Advances in neural information processing systems. 2013. 


Implement your own function to perform the analogy task (see [1] for concrete examples). Use the same distance metric as in [1]. Your function should be able to answer whether an analogy like: "a king is to a queen as a man is to a woman" ($e_{king} - e_{queen} + e_{woman} \approx e_{man}$) is true. 

In a perfect scenario, we would like that this analogy ($e_{king} - e_{queen} + e_{woman}$) results in the embedding of the word "man". However, it does not always result in exactly the same word embedding. The result of the formula is called the expected or the predicted word embedding. In this context, "man" is called the true or the actual word $t$. We want to find the word $p$ in the vocabulary, where the embedding of $p$ (referred to as $e_p$) is the closest to the predicted embedding (i.e. result of the formula). Then, we can check if $p$ is the same word as the true word $t$.  

You have to answer an analogy function using each embedding for  CBOW model. Show the true word (with distance similarity value between predicted embedding and true word embedding, i.e. `sim1`) , the predicted word (with distance similarity value between predicted embedding and the embedding of the word in the vocabulary that is closest to this predicted embedding, i.e. `sim2`).

Since `Alice in wonder land` is a relatively small dataset to training the model for the analogy task,  we may find it is difficult to get the correct answer by searching the nearest word embedding. Therefore, we extend the predicted word to the nearest `nr` number of words and check if the true word is within them. Do not expect the `nr` to be very small, it could be tens or hundreds.



<b>HINT</b>: to visualize the results of the analogy tasks , you can print them in a table. An example is given below.


| Analogy task | True word (sim1)  | Predicted nearaest word (sim2) | Correct? |True word within the top `nr` nearest word?|
|:------:|:------:|:------:|:------:|:------:|
|  queen is to king as woman is to ?	 | man (sim1) | predictd_word(sim2) | True / False |   True / False|

In [ ]:
analogy=('queen', 'king', 'woman', 'man')

First let us extract the embeddings of all vocabularies for the following task

In [ ]:
embeddings = model.embeddings(torch.Tensor([list(range(0,vocab_size))]).long().to(dev))
embeddings = embeddings.view(-1,100)
embeddings = embeddings.cpu().detach().numpy()

Then we define a 'get_nearest_words' function to retrive the nearest 'nr' words to a certain word embedding

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
def get_nearest_words(word_embed, ix_to_word,embeddings, nr=10):

    distances= cdist(word_embed[np.newaxis, :],embeddings, "cosine")[0]
    # Sort distances and store the indices of the `nr` lowest distances 
    top_sorted_indices = distances.argsort()[:nr]

    top_words = [ix_to_word[str(i)]    for i in top_sorted_indices]

    values = [distances[i] for i in top_sorted_indices]
    
    return list(zip(top_words, values))

Now let's search for the nearest `one` and `nr` words of the predicted word embedding, and print the analogy

In [ ]:
def print_analogy(analogy, embeddings, ix_to_word,word_to_ix, nr=10):
    # Retrieve the words from the analogy we need to compute
    word_a, word_b, word_c, word_true = analogy

    # Formulate the analogy task
    analogy_task = f"{word_a} is to {word_b} as {word_c} is to ?"

    print(f"Analogy Task: {analogy_task}")
    print("---------------------------------------------------")

    # Obtain embeddings for all the words
    embed_true =  embeddings[word_to_ix[word_true]]
    embed_a=embeddings[word_to_ix[word_a]]
    embed_b=embeddings[word_to_ix[word_b]]
    embed_c=embeddings[word_to_ix[word_c]]


    # Obtain the predicted embedding based on the analogy function
    embed_prediction = embed_b - embed_a + embed_c


    # The true word with distance similarity value between predicted embedding and true word embedding,
    # also denoted `sim1` in the text above
    sim1 = cdist(embed_true[np.newaxis, :],embed_prediction[np.newaxis, :],"cosine")[0][0]


    # The predicted word with distance similarity value between predicted embedding and the embedding of the word
    # in the vocabulary that is closest to this predicted embedding
    word_prediction, sim2 = get_nearest_words(embed_prediction, ix_to_word,embeddings)[0]

    nearest_words = get_nearest_words(embed_prediction, ix_to_word,embeddings,nr)

    # Print whether or not the true word was in the top nr 
    partially_correct = word_true in [word[0] for word in nearest_words]


    # Print all top nr words with their distance
    for word in nearest_words:
        print(f"{word[0]} => {word[1]:.4f}")
    print(f"Predicted: {word_prediction} ({sim2 :.4f}) - True: {word_true} ({sim1:.4f})")
    print(f"Correct? {word_prediction == word_true} - In the top {nr}? {partially_correct}")
    print("---------------------------------------------------")

In [ ]:
print_analogy(analogy, embeddings, ix_to_word,word_to_ix, nr=50)